# libCEED for Python examples

This is a tutorial to illustrate the main feautures of the Python interface for [libCEED](https://github.com/CEED/libCEED/), the low-level API library for efficient high-order discretization methods developed by the co-design [Center for Efficient Exascale Discretizations](https://ceed.exascaleproject.org/) (CEED) of the [Exascale Computing Project](https://www.exascaleproject.org/) (ECP).

While libCEED's focus is on high-order finite/spectral element method implementations, the approach is mostly algebraic and thus applicable to other discretizations in factored form, as explained in the [user manual](https://libceed.readthedocs.io/).

## Setting up libCEED for Python

Install libCEED for Python by running

In [ ]:
! python -m pip install libceed

## CeedQFunction

Here we show some basic examples to illustrate the `libceed.QFunction` class. In libCEED, QFunctions represent the spatial terms of the point-wise functions describing the physics at the quadrature points (see [the API documentation](https://libceed.readthedocs.io/en/latest/libCEEDapi.html#api-description)). As shown in the following sketch, QFunctions (such as the one depicted, which defines the Laplacian) are point-wise functions defined at quadrature points. Hence, QFunctions are independent from element shape, resolution and order.

![alt text][QFunctionSchematic]

[QFunctionSchematic]: ./img/QFunctionSketch.svg "Schematic of point-wise QFunctions, defined at quadrature points, belonging to elements that can have different shape, resolution and order."

* In the following example, we create and view two QFunctions (for the setup and apply, respectively, of the mass operator in 1D) from the gallery of available built-in QFunctions in libCEED

In [ ]:
import libceed
import numpy as np

ceed = libceed.Ceed()

qf_setup = ceed.QFunctionByName("Mass1DBuild")
qf_mass = ceed.QFunctionByName("MassApply")

print(qf_setup)
print(qf_mass)

* In the following example, we create and evaluate a built-in identity QFunction.

In [ ]:
qf = ceed.IdentityQFunction(1, libceed.EVAL_INTERP, libceed.EVAL_INTERP)

q = 8

u_array = np.zeros(q, dtype="float64")
for i in range(q):
  u_array[i] = i*i

u = ceed.Vector(q)
u.set_array(u_array, cmode=libceed.USE_POINTER)
v = ceed.Vector(q)
v.set_value(0)

inputs = [ u ]
outputs = [ v ]
qf.apply(q, inputs, outputs)

print('v =', v)

* In the following example, we create and evaluate a QFunction (for the mass operator in 1D) from the gallery of available built-in QFunctions in libCEED.

In [ ]:
qf_setup = ceed.QFunctionByName("Mass1DBuild")
qf_mass = ceed.QFunctionByName("MassApply")

q = 8

j_array = np.zeros(q, dtype="float64")
w_array = np.zeros(q, dtype="float64")
u_array = np.zeros(q, dtype="float64")
v_true  = np.zeros(q, dtype="float64")
for i in range(q):
  x = 2.*i/(q-1) - 1
  j_array[i] = 1
  w_array[i] = 1 - x*x
  u_array[i] = 2 + 3*x + 5*x*x
  v_true[i]  = w_array[i] * u_array[i]

j = ceed.Vector(q)
j.set_array(j_array, cmode=libceed.USE_POINTER)
w = ceed.Vector(q)
w.set_array(w_array, cmode=libceed.USE_POINTER)
u = ceed.Vector(q)
u.set_array(u_array, cmode=libceed.USE_POINTER)
v = ceed.Vector(q)
v.set_value(0)
qdata = ceed.Vector(q)
qdata.set_value(0)

inputs = [ j, w ]
outputs = [ qdata ]
qf_setup.apply(q, inputs, outputs)

inputs = [ w, u ]
outputs = [ v ]
qf_mass.apply(q, inputs, outputs)

print('v =', v)

* In the following example, we create and evaluate a built-in identity QFunction 3 fields per quadrature point.

In [ ]:
fields = 3

qf = ceed.IdentityQFunction(fields, libceed.EVAL_INTERP, libceed.EVAL_INTERP)

q = 8

u_array = np.zeros(q*fields, dtype="float64")
for i in range(q*fields):
  u_array[i] = i*i

u = ceed.Vector(q*fields)
u.set_array(u_array, cmode=libceed.USE_POINTER)
v = ceed.Vector(q*fields)
v.set_value(0)

inputs = [ u ]
outputs = [ v ]
qf.apply(q, inputs, outputs)

print('v =', v)